In [ ]:
#coding: utf-8
# py3.7
import numpy as np

import os
import matplotlib.pyplot as plt
import rasterio
from pathlib import Path  # 处理文件路径
import pandas as pd
import rasterio as rio


from rasterio.plot import show
import numpy as np
import matplotlib.pyplot as plt
from osgeo import gdal






def msa_change_pattern(lc_pre,lc_post,lanform,msa,center_x, center_y):
    '''
    previous
    '''
    human_distribution_pre = np.zeros_like(lc_pre)
    human_distribution_pre[lc_pre == 1] = 1
    human_distribution_pre[lc_pre == 8] = 2
    lc_pre = []
    '''
    post
    '''
    human_distribution_post = np.zeros_like(lc_post)
    human_distribution_post[lc_post == 1] = 1
    human_distribution_post[lc_post == 8] = 2
    lc_post = []

    lc_change_hill = (human_distribution_pre == 0) & (human_distribution_post == 1) & (lanform == 2)
    lc_change_plain = (human_distribution_pre == 0) & (human_distribution_post == 1) & (lanform == 1)

    msa_change=[]
    msa_change.append(np.sum(msa[(lc_change_hill > 0)&(msa>0)]))
    msa_change.append(np.sum(msa[(lc_change_plain > 0)&(msa>0)]))
    total = [center_x, center_y]  + msa_change
    return  np.asarray(total)



def process_dataset(dataset_path00,dataset_path10,dataset_path20,dataset_path_landform,dataset_path_slope,blocksize):

    glc00 = gdal.Open(dataset_path00)

    geotransform = glc00.GetGeoTransform()
    nodata_value1=glc00.GetRasterBand(1).GetNoDataValue()
    geopro, geotrans = geo_to_load(dataset_path00)
    '''
    glc10,glc20
    '''
    glc10 = gdal.Open(dataset_path10)
    nodata_value2 = glc10.GetRasterBand(1).GetNoDataValue()
    glc20 = gdal.Open(dataset_path20)
    nodata_value3 = glc20.GetRasterBand(1).GetNoDataValue()

    '''
    landfrom
    '''
    landform= gdal.Open(dataset_path_landform)
    nodata_value4 = landform.GetRasterBand(1).GetNoDataValue()
    '''
     dataset_path_slope
                        '''
    slope_dem = gdal.Open(dataset_path_slope)
    nodata_value5 = slope_dem.GetRasterBand(1).GetNoDataValue()


    width=min(glc00.RasterXSize,glc10.RasterXSize,glc20.RasterXSize,landform.RasterXSize)
    height = min(glc00.RasterYSize, glc10.RasterYSize, glc20.RasterYSize, landform.RasterYSize)
    print(width,height)

    total_blocks_x = width // blocksize+1
    total_blocks_y = height // blocksize+1

    datafolder='MSA_calulation'


    for i in range(blocksize*0, height, blocksize): # y
        for j in range(0, width, blocksize): #x

            x_size = min(blocksize, width - j)
            y_size = min(blocksize, height - i)


            data_block_landform = landform.ReadAsArray(j, i, x_size, y_size)
            data_block_landform[data_block_landform==nodata_value4]=0
            data_block_landform[data_block_landform>1]=2
            data_block_00 = glc00.ReadAsArray(j, i, x_size, y_size) // 10
            data_block_00[data_block_00 == nodata_value1] = 0

            data_block_10 = glc10.ReadAsArray(j, i, x_size, y_size) // 10
            data_block_10[data_block_10 == nodata_value2] = 0

            data_block_20 = glc20.ReadAsArray(j, i, x_size, y_size) // 10
            data_block_20[data_block_20 == nodata_value3] = 0
            human_expansion=(data_block_20==1)|(data_block_20==8)|(data_block_00==1)|(data_block_00==8)|(data_block_10==1)|(data_block_10==8)
            if np.max(human_expansion) > 0:

                block_x = j // blocksize
                block_y = i // blocksize



                data_block_msa = slope_dem.ReadAsArray(j, i, x_size, y_size)
                data_block_msa[data_block_msa == nodata_value5] = 0
                data_block_msa=data_block_msa

                center_x = geotransform[0] + (j + blocksize / 2) * geotransform[1]
                center_y = geotransform[3] + (i + blocksize / 2) * geotransform[5]
                print(j,i,'------',center_x, center_y)

                slope_10_20=msa_change_pattern(data_block_10,data_block_20,data_block_landform,data_block_msa,center_x, center_y)
                with open(os.path.join(datafolder,'cropland_msa_10_20.csv'), 'a') as file:

                    file.write(','.join(map(str, slope_10_20)) + '\n')

                slope_00_10=msa_change_pattern(data_block_00,data_block_10,data_block_landform,data_block_msa,center_x, center_y)
                with open(os.path.join(datafolder,'cropland_msa_00_10.csv'), 'a') as file:

                    file.write(','.join(map(str, slope_00_10)) + '\n')
                slope_00_20=msa_change_pattern(data_block_00,data_block_20,data_block_landform,data_block_msa,center_x, center_y)
                with open(os.path.join(datafolder,'cropland_msa_00_20.csv'), 'a') as file:

                    file.write(','.join(map(str, slope_00_20)) + '\n')




    dataset = None


if __name__ == "__main__":
    dataset_path00 = r"GLC30\GLCV00.tif"
    dataset_path10 = r"GLC30\GLCV10.tif"
    dataset_path20 = r"GLC30\GLCV20.tif"
    dataset_path_landform = r"E:\GBLU\GBLU.tif"
    dataset_path_MSA = r"Globio4_TerrestrialMSA_wbvert_10sec_2015\MSA30.tif"


    process_dataset(dataset_path00, dataset_path10, dataset_path20, dataset_path_landform,dataset_path_MSA,blocksize=167)